In [2]:
import pandas as pd
wine = pd.read_csv('https://bit.ly/wine_csv_data')

In [3]:
data = wine[['alcohol','sugar', 'pH']].to_numpy()
target = wine[['class']]

print(data.shape, target.shape)

(6497, 3) (6497, 1)


In [4]:
from sklearn.model_selection import train_test_split

train_x, train_y, target_x, target_y = train_test_split(data, target, test_size=0.2, 
                                                        random_state=42)


In [5]:
sub_input, val_input, sub_target, val_target = train_test_split(train_x, target_x,
                                                                test_size=0.2, random_state=42)

In [6]:
print(sub_input.shape, val_input.shape)
print(train_y.shape, target_y.shape)

(4157, 3) (1040, 3)
(1300, 3) (1300, 1)


In [7]:
from sklearn.tree import DecisionTreeClassifier
dt = DecisionTreeClassifier(random_state=42)
dt.fit(sub_input, sub_target)
print(dt.score(sub_input, sub_target))
print(dt.score(val_input, val_target))

0.9971133028626413
0.864423076923077


In [8]:
from sklearn.model_selection import cross_validate

scores = cross_validate(dt, train_x, target_x)
print(scores)

{'fit_time': array([0.0079968 , 0.00799775, 0.0080018 , 0.00799751, 0.00799608]), 'score_time': array([0.00100279, 0.00099921, 0.00100279, 0.00100327, 0.00100207]), 'test_score': array([0.86923077, 0.84615385, 0.87680462, 0.84889317, 0.83541867])}


In [9]:
import numpy as np
print(np.mean(scores['test_score']))

0.855300214703487


In [10]:
from sklearn.model_selection import StratifiedKFold
scores = cross_validate(dt, train_x, target_x, cv=StratifiedKFold())
print(np.mean(scores['test_score']))

0.855300214703487


In [11]:
splitter = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
scores = cross_validate(dt, train_x, target_x, cv=splitter)
print(np.mean(scores['test_score']))

0.8574181117533719


In [14]:
from sklearn.model_selection import GridSearchCV

params = {'min_impurity_decrease' : [0.0001, 0.0002, 0.0003, 0.0004, 0.0005]}
gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params,n_jobs= -1)

In [15]:
gs.fit(train_x, target_x)
dt = gs.best_estimator_
print(dt.score(train_y, target_y))

0.8653846153846154
